In [256]:
import pandas as pd
# import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import json
import folium
warnings.filterwarnings("ignore")
# 한글 폰트 문제 해결 
# matplotlib은 한글 폰트를 지원하지 않음
# os정보
import platform

# font_manager : 폰트 관리 모듈
# rc : 폰트 변경 모듈
from matplotlib import font_manager, rc
# unicode 설정
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':
    rc('font', family='AppleGothic') # os가 macos
elif platform.system() == 'Windows':
    path = 'c:/Windows/Fonts/malgun.ttf' # os가 windows
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print("Unknown System")

통계청,「국내인구이동통계」, 2022.12, 2023.02.09, 전출지/전입지(시도)/이동규모별 이동건수\
https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B26008&conn_path=I2

In [257]:
move1 = pd.read_csv('../../Data/people/전출지:전입지(시도):이동규모별 이동건수07_11.csv', encoding='euc-kr')#, index_col='시점')
move2 = pd.read_csv('../../Data/people/전출지:전입지(시도):이동규모별 이동건수12_17.csv', encoding='euc-kr')#, index_col='시점')
move3 = pd.read_csv('../../Data/people/전출지:전입지(시도):이동규모별 이동건수18_22.csv', encoding='euc-kr')#, index_col='시점')
move2.head(15)

,이동규모별,시점,전출지별,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
0,1인,2012,서울특별시,363976,10915,6698,23409,6887,8369,3920,1102,150870,14194,9286,14021,10886,10253,10387,10056,4204
1,1인,2012,부산광역시,15969,117890,3218,1957,886,1405,7373,179,11110,1442,1166,2179,1129,2207,6812,30073,1244
2,1인,2012,대구광역시,10172,3197,75990,1218,415,1131,2544,120,8169,1088,1139,1626,590,639,25392,5682,464
3,1인,2012,인천광역시,19125,1620,1000,90540,1164,1603,688,238,30619,3329,2312,4908,2336,2363,2345,1838,804
4,1인,2012,광주광역시,8660,885,364,1357,54111,872,387,94,7535,530,559,1138,3643,20786,593,916,483
5,1인,2012,대전광역시,9766,1121,918,1669,682,49402,518,2227,8556,1088,4775,9847,2346,993,1805,1345,312
6,1인,2012,울산광역시,4957,5755,1870,660,246,529,33861,29,3441,612,501,824,513,555,4562,4353,257
7,1인,2012,세종특별자치시,721,86,62,150,55,1163,39,1228,821,98,960,1054,117,92,122,107,22
8,1인,2012,경기도,130083,8335,6123,31618,6503,8287,3326,1436,398506,17829,12809,20116,11624,10588,11663,9207,3466
9,1인,2012,강원도,15039,1346,1087,3415,499,1269,737,139,17862,54829,2713,1901,889,800,2543,1463,350


#### 2007 ~ 2022 년 DataFrame 합치기

In [258]:
move = pd.concat([move1,move2,move3], axis=0)

In [259]:
print(move1.shape)
print(move2.shape)
print(move3.shape)
move.shape

(800, 19)
(1020, 20)
(850, 20)


(2670, 20)

In [260]:
move.head(2)

,이동규모별,시점,전출지별,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,세종특별자치시
0,1인,2007,서울특별시,487525,11292,6600,25826,7463,8330,3252,186208,13743,9693,17628,12789,12023,10779,10356,3606,NaN
1,1인,2007,부산광역시,18880,128899,3097,2434,840,1429,7640,13326,1545,1241,2479,1153,2385,7500,34963,1360,NaN


In [261]:
move.tail(2)

,이동규모별,시점,전출지별,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,세종특별자치시
848,10인이상,2022,경상남도,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0.0
849,10인이상,2022,제주특별자치도,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0.0


In [262]:
# column check
print(move.columns)
print(len(move.columns))

Index(['이동규모별', '시점', '전출지별', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시',
       '대전광역시', '울산광역시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도', '세종특별자치시'],
      dtype='object')
20


#### 이동규모별을 숫자로 치환한 뒤, 각 시도에 규모별을 곱하여 인구수로 바꾸기

In [263]:
move['이동규모별'].unique()

array(['1인', '2인', '3인', '4인', '5인', '6인', '7인', '8인', '9인', '10인이상'],
      dtype=object)

In [264]:
before=['1인', '2인', '3인', '4인', '5인', '6인', '7인', '8인', '9인', '10인이상']
after = [1,2,3,4,5,6,7,8,9,10]
move.replace(before,after, inplace=True)
move['이동규모별'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [265]:
move.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2670 entries, 0 to 849
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   이동규모별    2670 non-null   int64  
 1   시점       2670 non-null   int64  
 2   전출지별     2670 non-null   object 
 3   서울특별시    2670 non-null   int64  
 4   부산광역시    2670 non-null   int64  
 5   대구광역시    2670 non-null   int64  
 6   인천광역시    2670 non-null   int64  
 7   광주광역시    2670 non-null   int64  
 8   대전광역시    2670 non-null   int64  
 9   울산광역시    2670 non-null   int64  
 10  경기도      2670 non-null   int64  
 11  강원도      2670 non-null   int64  
 12  충청북도     2670 non-null   int64  
 13  충청남도     2670 non-null   int64  
 14  전라북도     2670 non-null   int64  
 15  전라남도     2670 non-null   int64  
 16  경상북도     2670 non-null   int64  
 17  경상남도     2670 non-null   int64  
 18  제주특별자치도  2670 non-null   int64  
 19  세종특별자치시  1870 non-null   float64
dtypes: float64(1), int64(18), object(1)
memory usage: 438

#### NaN 값이 있는 세종특별자치시 0으로 바꾸기

In [266]:
move.fillna(0, inplace=True)

In [267]:
# 세종특별자치시 float -> int 변환
move['세종특별자치시'] = move['세종특별자치시'].astype(int)
move.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2670 entries, 0 to 849
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   이동규모별    2670 non-null   int64 
 1   시점       2670 non-null   int64 
 2   전출지별     2670 non-null   object
 3   서울특별시    2670 non-null   int64 
 4   부산광역시    2670 non-null   int64 
 5   대구광역시    2670 non-null   int64 
 6   인천광역시    2670 non-null   int64 
 7   광주광역시    2670 non-null   int64 
 8   대전광역시    2670 non-null   int64 
 9   울산광역시    2670 non-null   int64 
 10  경기도      2670 non-null   int64 
 11  강원도      2670 non-null   int64 
 12  충청북도     2670 non-null   int64 
 13  충청남도     2670 non-null   int64 
 14  전라북도     2670 non-null   int64 
 15  전라남도     2670 non-null   int64 
 16  경상북도     2670 non-null   int64 
 17  경상남도     2670 non-null   int64 
 18  제주특별자치도  2670 non-null   int64 
 19  세종특별자치시  2670 non-null   int64 
dtypes: int64(19), object(1)
memory usage: 438.0+ KB


컬럼 값 곱하기

In [268]:
testmove = move.copy()

In [269]:
print(len(testmove.index))
print(len(testmove.columns))

2670
20


In [270]:
for i in range(0,len(testmove.index)):
   for j in range(3,len(testmove.columns)):
    testmove.iloc[i,j] = testmove.iloc[i,j] * testmove.iloc[i,0]

In [271]:
testmove = testmove[['시점', '전출지별', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시',
       '대전광역시', '울산광역시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도',
       '경상남도', '제주특별자치도', '세종특별자치시']]

In [272]:
testmove.head()

,시점,전출지별,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,세종특별자치시
0,2007,서울특별시,487525,11292,6600,25826,7463,8330,3252,186208,13743,9693,17628,12789,12023,10779,10356,3606,0
1,2007,부산광역시,18880,128899,3097,2434,840,1429,7640,13326,1545,1241,2479,1153,2385,7500,34963,1360,0
2,2007,대구광역시,10981,3120,87665,1541,368,1253,2391,8814,1209,1153,1626,551,609,29607,6378,430,0
3,2007,인천광역시,23480,1816,1116,115286,1398,1520,651,35911,3300,2408,6384,2787,2830,2314,2205,595,0
4,2007,광주광역시,11031,721,364,1776,60950,895,303,9179,543,546,1192,3646,25378,530,992,568,0


In [273]:
testmove2 = testmove.copy()
testmove.set_index('시점', inplace=True, drop=True)
testmove2.set_index('전출지별', inplace=True, drop=True)
testmove.head()

,전출지별,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,세종특별자치시
시점,,,,,,,,,,,,,,,,,,
2007,서울특별시,487525,11292,6600,25826,7463,8330,3252,186208,13743,9693,17628,12789,12023,10779,10356,3606,0
2007,부산광역시,18880,128899,3097,2434,840,1429,7640,13326,1545,1241,2479,1153,2385,7500,34963,1360,0
2007,대구광역시,10981,3120,87665,1541,368,1253,2391,8814,1209,1153,1626,551,609,29607,6378,430,0
2007,인천광역시,23480,1816,1116,115286,1398,1520,651,35911,3300,2408,6384,2787,2830,2314,2205,595,0
2007,광주광역시,11031,721,364,1776,60950,895,303,9179,543,546,1192,3646,25378,530,992,568,0


In [274]:
pd.pivot_table(data=testmove, values=testmove[['서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시',
       '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도',
       '세종특별자치시']], aggfunc=sum, index=testmove['전출지별'] )

,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,세종특별자치시,울산광역시,인천광역시,전라남도,전라북도,제주특별자치도,충청남도,충청북도
전출지별,,,,,,,,,,,,,,,,,
강원도,2286334,461666,41268,59290,12649,26578,44567,33460,336838,7215,15543,79204,23517,22423,11502,57350,74621
경기도,484988,21596082,246532,286905,150176,157259,239730,220940,4193904,73256,76216,1134247,259644,292174,130805,627526,375559
경상남도,43301,286144,4704740,158449,22385,142163,44762,772745,289790,9155,126438,54615,74822,35485,29092,64917,39143
경상북도,60867,340164,149478,3254593,12788,696151,56863,150712,279911,9754,122514,61777,28289,23269,17337,72119,68427
광주광역시,14131,181732,22854,13600,2591347,9553,24547,19840,188763,7410,6768,34426,572946,89191,15170,31249,14680
대구광역시,29361,215588,142216,738899,9726,4088619,33885,82818,225100,7660,53903,32746,15483,14759,17284,41490,31404
대전광역시,39142,260992,37140,45966,19756,26970,2420984,30649,238700,176684,11867,42601,27791,63994,11257,284469,135803
부산광역시,35574,281668,925101,159378,19554,76995,36684,5623194,342906,8402,170074,50569,55141,25873,37523,53311,30946
서울특별시,343608,5847392,220610,224784,145295,151867,202934,248101,17733223,59434,76872,722755,222700,236168,128992,357236,225101


In [275]:
testmove.index

Int64Index([2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007, 2007,
            ...
            2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022, 2022],
           dtype='int64', name='시점', length=2670)

In [276]:
# pd.multiindex.from
multimove = testmove.copy()
multimove
multimove.set_index([multimove.index, multimove.전출지별], inplace=True, drop=True)
multimove.drop(columns='전출지별', inplace=True)

In [277]:
# testmove.xs('서울특별시', axis=1)

In [278]:
testmove

,전출지별,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,세종특별자치시
시점,,,,,,,,,,,,,,,,,,
2007,서울특별시,487525,11292,6600,25826,7463,8330,3252,186208,13743,9693,17628,12789,12023,10779,10356,3606,0
2007,부산광역시,18880,128899,3097,2434,840,1429,7640,13326,1545,1241,2479,1153,2385,7500,34963,1360,0
2007,대구광역시,10981,3120,87665,1541,368,1253,2391,8814,1209,1153,1626,551,609,29607,6378,430,0
2007,인천광역시,23480,1816,1116,115286,1398,1520,651,35911,3300,2408,6384,2787,2830,2314,2205,595,0
2007,광주광역시,11031,721,364,1776,60950,895,303,9179,543,546,1192,3646,25378,530,992,568,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022,전라북도,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2022,전라남도,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0
2022,경상북도,0,0,0,0,0,0,0,0,0,0,0,0,10,20,0,0,0


In [279]:
multimove

서울특별시   부산광역시  대구광역시   인천광역시  광주광역시  대전광역시  울산광역시     경기도  \
시점   전출지별                                                                  
2007 서울특별시    487525   11292   6600   25826   7463   8330   3252  186208   
     부산광역시     18880  128899   3097    2434    840   1429   7640   13326   
     대구광역시     10981    3120  87665    1541    368   1253   2391    8814   
     인천광역시     23480    1816   1116  115286   1398   1520    651   35911   
     광주광역시     11031     721    364    1776  60950    895    303    9179   
...              ...     ...    ...     ...    ...    ...    ...     ...   
2022 전라북도          0       0      0       0      0      0      0       0   
     전라남도          0       0      0       0      0      0      0      10   
     경상북도          0       0      0       0      0      0      0       0   
     경상남도          0       0      0       0      0      0      0       0   
     제주특별자치도       0       0      0       0      0      0      0       0   

                강원도  충청북도   충청남도   전라북도   전라남도   경상북도   경상남도  제주특별자치도  세종특별자치시  
시점   전출지별                                                                       
2007 서울특별시    13743  9693  17628  12789  12023  10779  10356     3606        0  
     부산광역시     1545  1241   2479   1153   2385   7500  34963     1360        0  
     대구광역시     1209  1153   1626    551    609  29607   6378      430        0  
     인천광역시     3300  2408   6384   2787   2830   2314   2205      595        0  
     광주광역시      543   546   1192   3646  25378    530    992      568        0  
...             ...   ...    ...    ...    ...    ...    ...      ...      ...  
2022 전라북도         0     0      0      0      0      0      0        0        0  
     전라남도         0     0      0      0      0      0      0        0        0  
     경상북도         0     0      0      0     10     20      0        0        0  
     경상남도         0     0      0      0      0      0     50        0        0  
     제주특별자치도      0     0      0      0      0      0      0       30        0  

[2670 rows x 17 columns]

In [286]:
testmove2[testmove2['시점']==2012]

,시점,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,세종특별자치시
전출지별,,,,,,,,,,,,,,,,,,
서울특별시,2012,363976,10915,6698,23409,6887,8369,3920,150870,14194,9286,14021,10886,10253,10387,10056,4204,1102
부산광역시,2012,15969,117890,3218,1957,886,1405,7373,11110,1442,1166,2179,1129,2207,6812,30073,1244,179
대구광역시,2012,10172,3197,75990,1218,415,1131,2544,8169,1088,1139,1626,590,639,25392,5682,464,120
인천광역시,2012,19125,1620,1000,90540,1164,1603,688,30619,3329,2312,4908,2336,2363,2345,1838,804,238
광주광역시,2012,8660,885,364,1357,54111,872,387,7535,530,559,1138,3643,20786,593,916,483,94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
전라북도,2012,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0,0,0
전라남도,2012,0,0,0,0,10,0,0,0,0,0,0,0,10,0,0,0,0
경상북도,2012,0,0,0,0,0,0,0,0,0,0,0,0,0,30,0,0,0
